# Peparacion del data frame

## Configuración inicial y carga de datos

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)

Cargar archivo

In [4]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    name=fn
df = pd.read_csv(name, sep=";")


Saving Anexo ET_demo_round_traces_2022.csv to Anexo ET_demo_round_traces_2022.csv


Limpiar nulos

In [5]:
df.dropna(how='any', inplace=True)
null_values_new = df.isnull().sum()
print(null_values_new[null_values_new > 0])

Series([], dtype: int64)


Quitar variables con las que no se van a usar en el modelo

In [6]:
droped_vars = ("AbnormalMatch","TravelledDistance","Map","InternalTeamId","MatchId",
               "RoundId","RoundWinner","Survived","RLethalGrenadesThrown","RNonLethalGrenadesThrown",
               "PrimaryAssaultRifle","PrimarySniperRifle","PrimaryHeavy","PrimarySMG","PrimaryPistol",
               "RoundAssists","RoundHeadshots","RoundFlankKills","MatchFlankKills","MatchAssists","MatchHeadshots",
               "MatchKills","Unnamed: 0")

for var in droped_vars:
  df = df.drop(var, axis=1)


transformar objeto a int

In [7]:
# Transformar variables Objecto a int
try:
  df['TimeAlive'] = pd.to_numeric(df['TimeAlive'].str.replace('.', ''))
  df['FirstKillTime'] = pd.to_numeric(df['FirstKillTime'].str.replace('.', ''))
except Exception as e:
  print(f'{e}\n')


Transformar las variables categoricas Team y MatchWinner a binario.

Team:
* Terrorist: 0
* CounterTerrorist: 1

MatchWinner:
* True: 1
* False: 0

In [8]:
df.Team.replace(to_replace=['Terrorist', 'CounterTerrorist'], value=[0, 1], inplace=True )
df.MatchWinner.replace(to_replace=['False', 'True'], value=[0, 1], inplace=True )
df.MatchWinner = df.MatchWinner.astype(int)

In [9]:
df.head()

,Team,MatchWinner,TimeAlive,FirstKillTime,RoundKills,RoundStartingEquipmentValue,TeamStartingEquipmentValue
0,0,1,51120248995704500,0,0,750,4400
1,0,1,4348662552266170,0,0,800,4400
2,0,1,3735469847054540,0,0,1000,4400
3,0,1,4767886136441470,0,0,850,4400
4,0,1,5312251089027310,0,0,1000,4400


## Objetivo

Utilizar las variables para determinar el tiempo de vida (TimeAlive) del jugador

# SVM

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import sklearn.gaussian_process # Kernel de transformación del espacio
import sklearn.preprocessing

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split

In [11]:
X_vals = ("Team","MatchWinner","FirstKillTime","RoundKills",
          "RoundStartingEquipmentValue","TeamStartingEquipmentValue")

def svm_LinearScoring(x, y):
  print(f'--- X Value: {x}')

  x = np.array(df[x]).reshape(-1, 1)
  y = np.array(df[y]).reshape(-1, 1)

  Xtrain, Xtest, Ytrain, Ytest = train_test_split(x,y,test_size=0.2, random_state = 101)

  model = SVC()
  model.fit(Xtrain, Ytrain)

  predictions = model.predict(Xtest)
  print(classification_report(Ytest, predictions))

In [12]:
df["TimeAlive"] = df.TimeAlive.div(10**10).round(0)
df["TimeAlive"] = df.TimeAlive.astype(int)
df["TimeAlive"].head()

,TimeAlive
0,5112025
1,434866
2,373547
3,476789
4,531225


In [13]:
X_train, X_test, y_train, y_test = train_test_split(
						np.array(df.drop(["TimeAlive",
                        "Team","MatchWinner","FirstKillTime","RoundKills"],axis=1)),
                        np.array(df["TimeAlive"]),
				test_size = 0.30, random_state = 101)

In [ ]:
model = SVC()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10, 100, 1000],
			'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
			'kernel': ['rbf']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

print(grid.best_estimator_)

In [ ]:
grid_predictions = grid.predict(X_test)

print(classification_report(y_test, grid_predictions))

In [ ]:
'''
for val in X_vals:
  svm_LinearScoring(val, "TimeAlive")
  print('')
'''